In [ ]:
import glob
import os

import pandas as pd
from pandas.api.types import is_string_dtype
from dask import compute
from dask.delayed import delayed
import dask.dataframe as dd
import zipfile

from config import ERCOT_RAW_ROOT, ERCOT_GOLD_ROOT

In [ ]:
filenames = glob.glob(f"{ERCOT_RAW_ROOT}/*.zip", recursive=True)
filenames[0:3]

In [ ]:
def to_year(filename):   
    return int(filename.split("_")[-1].replace(".zip", ""))
    
raw_file = '../data/raw/ercot\\native_Load_2016.zip'
result = to_year(raw_file)
result

In [ ]:
def to_hour_beginning(hour_ending):
    l = hour_ending.str[0:-5]
    m = hour_ending.str[-5:-3]
    m = m.astype(int)-1
    m = m.astype(str).str.zfill(2)
    r = hour_ending.str[-3:0]
    pd.to_datetime(l + m + r)
    return l + m + r
    

value = pd.DataFrame({"Hour Ending": ['01/01/2021 01:00']})
to_hour_beginning(value["Hour Ending"])

In [ ]:
schema = {
    "Hour Ending Raw": 'object',
    "Hour Ending": 'datetime64[ns]',
    "Hour Beginning": 'datetime64[ns]',
    "COAST": 'float',
    "EAST": "float",
    "FWEST": "float",
    "WEST": "float",
    "NORTH": "float",
    "NCENT": "float",
    "SOUTH": "float",
    "SCENT": "float",
    "WEST": "float",
    "ERCOT": "float",
    "year": "int",
}
rename_columns = {
    "Hour_End": "Hour Ending",
    "HourEnding": "Hour Ending",
    "FAR_WEST": "FWEST",
    "NORTH_C": "NCENT",
    "SOUTHERN": "SOUTH",
    "SOUTH_C": "SCENT",
}
columns = list(schema.keys())

In [ ]:
def read_archive(filename):
    archive = zipfile.ZipFile(filename, 'r')
    filename_xlsx = archive.filelist[0].filename
    xlfile = archive.open(filename_xlsx)
    df = (
        pd
        .read_excel(xlfile)
        .dropna()
        .rename(columns=rename_columns)
    )
    
    year = to_year(filename)
    df["year"]=year
    
    df["Hour Ending Raw"] = df['Hour Ending'].astype(str)
    if is_string_dtype(df['Hour Ending']):
        date = df["Hour Ending"].str[0:10]
        hour = (df["Hour Ending"].str[11:13].astype(int)-1).astype(str).str.zfill(2)
        df["Hour Beginning"] = pd.to_datetime(date + ' ' + hour)
        df["Hour Ending"] = df["Hour Beginning"] + pd.Timedelta(hours=1)
    else:
        df["Hour Ending"] = df['Hour Ending'].dt.round('min')
        df["Hour Beginning"] = df['Hour Ending'].dt.round('min')-pd.Timedelta(hours=1)
    return df[columns]


dfs = [delayed(read_archive)(fn) for fn in filenames]
df = dd.from_delayed(dfs, meta=schema)  # df is a dask dataframe
df.head()

In [ ]:
id_vars=['year', 'Hour Beginning', "Hour Ending", "Hour Ending Raw"]
value_vars = list(set(df.columns)-set(id_vars))
df_normalized = dd.melt(df, id_vars=id_vars, value_vars=value_vars).rename(columns={"variable": "area", "value": "load"})
df_normalized.head()

In [ ]:
gold_path = f"{ERCOT_GOLD_ROOT}.parquet"
df_normalized.to_parquet(gold_path, partition_on=["area", "year"])

Previous: [Transform NOAA Observations](transform_noaa_observations.ipynb)
Next: [Transform to Training Data Set](transform_to_training_dataset.ipynb)